## Bibliotecas

In [1]:
import pandas as pd
import numpy as np
import scipy as scp
from bs4 import BeautifulSoup
import requests as rq
import re
from requests.exceptions import HTTPError
import urllib
import urllib.request

from selenium import webdriver
from reppy.robots import Robots
from urllib.parse import urlparse
import time
import operator
#import nltk

## Crawler

In [66]:
Dominio = ['https://scryfall.com/','http://www.mtgmintcard.com/',
           'https://www.cardkingdom.com/','http://www.starcitygames.com/',
           'https://www.capefeargames.com/','https://www.mtgotraders.com/',
           'https://www.wizardscupboard.com/','https://abugames.com/']
Dados = []
TempoDominio = []

In [57]:
# Função para marcar a page como visitada.
def Visitou (url,paginasVisitadas):
    if url not in paginasVisitadas:
        return False
    else:
        return True

In [58]:
#Função para verificar se a requisição sofreu algum problema.
def verificarRequest(url):
    try:
        response = rq.head(url,allow_redirects=True)
        ##print(response)
        if response.status_code == rq.codes.ok:
            return True
        else:
            return False
    except:
        return False

In [59]:
def CrawlerDinâmico(url):
    #C:\\Users\\Usuário\\Downloads\\chromedriver76\\chromedriver.exe 
    driver = webdriver.Chrome("C:\\Users\\Usuário\\Downloads\\chromedriver76\\chromedriver.exe")
    driver.get(url)
    res = driver.execute_script("return document.documentElement.outerHTML")
    driver.quit
    return res

In [60]:
def verificarRobotTxt(url):
    robots = Robots.fetch(url + '/robots.txt')
    return robots.allowed(url, '*')

In [61]:
# Criar uma Tabela com as informações sobre links.
def Tabela(count,url,site):
    Dados.append(dict(zip(['Número','Link','Site'],[count,url,site])))

## Padrões

In [62]:
def Words():
    goods = ['magic','single','set','catalog','gathering','card','store','foil','product','buylist']
    bad = ['pokemom','yugioh']
    words = [goods,bad]    
    return words

In [63]:
def Padroes():
    
    regex = r'(.+)(magic_singles-)(.+)(\/)(.+)(\/)(\d+)'
    CapeFearGamesMagicCard = re.compile(regex)
    
    regex2 = r'(.+)(store\/)([A-Z]{3}|[A-Z]{1}[0-9]{2})(.+)'
    MtGoTradersMagicCard = re.compile(regex2)
    # |||| (.)*(p-)(\d+)(\.html)
    regex3 = r'(.+)(-p-)(\d+)(\.html)'
    WizardCupBoardMagicCard = re.compile(regex3)
    
    regex4 = '(.)*(magic-the-gathering\/)(singles\/)(product-detail\/)(.+)'
    AbuGamesMagicCard = re.compile(regex4)

    regex5 = r'(.+)(mtg\/)(.+)'
    CardKingdomMagicCard = re.compile(regex5)

    regex6 = r'(.+)(magic_the_gathering\/)(product\/)(\d+)(\/\d+)*'
    StarCityMagicCard = re.compile(regex6)

    regex7 = r'(.+)(card)(.+)(\d+)(.+)'
    ScryfallMagicCard = re.compile(regex7)

    regex8 = r'(.+)(mtg\/)(singles\/)(.+\/)(.+\/)(.+)'
    MtgMIntCard = re.compile(regex8)
    
    padroesLinks = [CapeFearGamesMagicCard,CardKingdomMagicCard,AbuGamesMagicCard,ScryfallMagicCard,
                    StarCityMagicCard,WizardCupBoardMagicCard,MtGoTradersMagicCard,MtgMIntCard]
    return padroesLinks

In [64]:
def Score(url,siteInicial,texto):
    score = 0
    
    regexMoney = re.compile(r'(\$)(\d+)(.)(\d{2})')
    
    # Dentro do Dominio +1
    if siteInicial in url:
        score = score + 1
        
    # url contem alguma palavra da lista de palavraes boas que aparecem no link relevante +2   
    for palavra in Words()[0]:
        #casefold ignorar sensitive-case
        if palavra.casefold() in url.casefold():
            score = score + 2
            
    # url fullmatch com link relevante +50
    for padrao in Padroes():
        if padrao.fullmatch(url):
            score = score + 5
    
    #CArdpage tem texto sobre valor da carta. +100
    if regexMoney.match(texto):
        score = score + 100
    
     # url contem alguma palavra da lista de palavras ruins(pokemom, yugioh ...) zera o score.  
    for palavra in Words()[1]:
        if palavra.casefold() in url.casefold():
            score = 0
            return score
    return score

## * Heurística

In [67]:
def CardsCrawler(url):
    print('Dominio: ' + url.geturl())
    count = 0
    score = 0
    paginas = [(url,score)]
    paginasVisitadas = []
    robots = Robots.fetch(url.geturl() + '/robots.txt')
    while count < 1000 and len(paginas) > 0:
        #Ordenar a busca pelo Score.
        paginas.sort(key=operator.itemgetter(1),reverse=True)
        
        primeiraPagina = paginas.pop(0)
        
        # Verifica se a pagina foi visitada.Se foi, procuro o proximo link da lista que não foi visitado.
        # Verifico se o hostname de cada pagina é igual ao do dominio, se não, rocuro o proximo link da lista (navegar apenas dentro do site)
        # Verifico se hostname existe , se não , procuro o proximo link que tenha.
        # Verifico se é possivel crawlear a pagina de acordo com as regras do robot.txt , se não , procuro o proximo link.
        # Verifico se a respota da requisição funciona
        while Visitou(primeiraPagina[0].geturl(),paginasVisitadas) == True or primeiraPagina[0].hostname != url.hostname or primeiraPagina[0].hostname == None or robots.allowed(primeiraPagina[0].geturl(), '*') == False or verificarRequest(primeiraPagina[0].geturl()) == False:
            if len(paginas) == 0:
                print('lascou@!')
                return
            primeiraPagina = paginas.pop(0)
    
        # Verficar se o site é dinamico.
        if (Dominio[7] in url.geturl()):
            texto = CrawlerDinâmico(primeiraPagina[0].geturl())
        else:
            #request - usando o content type
            codigo_fonte = rq.get(primeiraPagina[0].geturl(),headers={'content-type': 'text/html'})
            # texto do codigo_fonte.
            texto = codigo_fonte.text
        soup = BeautifulSoup(texto,'html.parser')

        # Encontrar todos os links da pagina(url) filtrando pelas tags li ul tr td.
        for lista in soup.findAll(re.compile(r'li|ul|td|tr|div')):
            for link in lista.findAll('a',href = True):
                
                href = link.get('href')
                # Correção de href relativos para absolutos.
                href = urllib.parse.urljoin(url.geturl(),href)
                
                #Texto na ancora.
                ancoraTexto = link.text
                
                #Texto no Parent da ancora
                parent_text = re.sub('[\s+]', '', link.parent.text)
                if len(parent_text) > 200:
                    parent_text = ''
                
                #concatenar textos da ancora e da tag pai.
                urltexto = ancoraTexto + parent_text

                paginas.append((urlparse(href),Score(href,url.geturl(),urltexto)))
            
        # Marco como visitada a urlparse    
        paginasVisitadas.append(primeiraPagina[0].geturl())
        
        #esperar 500 milisegundos para visitar o prox link.
        time.sleep(.5)
        
        count = count + 1
        print(count,'-',primeiraPagina[0].geturl(),'Score: ',primeiraPagina[1],' Qtd Paginas=', len(paginas))
        
        # Salvar a page localmente
        with open("Paginas/PaginasHeuristica/"+str(url.hostname)+str(count)+".html", 'w' , encoding='utf-8') as outfile:
            outfile.write(texto)
        
        Tabela(count,primeiraPagina[0].geturl(),url.hostname)

### Download pages do Dominio.

In [ ]:
for url in Dominio:
    inicio = time.time()
    CardsCrawler(urlparse(url))
    fim = time.time()
    tempo = fim - inicio
    print('Tempo do Site: ',tempo, 'Site: ', url)
    TempoDominio.append(dict(zip(['Dominio','Tempo'],[url,tempo])))

Dominio: https://scryfall.com/
1 - https://scryfall.com/ Score:  0  Qtd Paginas= 178
2 - https://scryfall.com/card/m15/219/hot-soup Score:  8  Qtd Paginas= 520
3 - https://scryfall.com/card/som/153/echo-circlet Score:  8  Qtd Paginas= 846
4 - https://scryfall.com/card/soi/262/shard-of-broken-glass Score:  8  Qtd Paginas= 1182
5 - https://scryfall.com/card/dtk/239/gate-smasher Score:  8  Qtd Paginas= 1521
6 - https://scryfall.com/card/5dn/129/healers-headdress Score:  8  Qtd Paginas= 1842
7 - https://scryfall.com/card/wwk/20/stoneforge-mystic Score:  8  Qtd Paginas= 2249
8 - https://scryfall.com/card/wwk/129/razor-boomerang Score:  8  Qtd Paginas= 2578
9 - https://scryfall.com/card/m15/219/es/sopa-caliente Score:  8  Qtd Paginas= 2886
10 - https://scryfall.com/card/m15/219/fr/soupe-bouillante Score:  8  Qtd Paginas= 3222
11 - https://scryfall.com/card/m15/219/de/hei%C3%9Fe-suppe Score:  8  Qtd Paginas= 3558
12 - https://scryfall.com/card/m15/219/it/zuppa-bollente Score:  8  Qtd Paginas=

77 - https://scryfall.com/card/wwk/129/de/messerscharfer-bumerang Score:  8  Qtd Paginas= 24405
78 - https://scryfall.com/card/wwk/129/it/lama-boomerang Score:  8  Qtd Paginas= 24739
79 - https://scryfall.com/card/wwk/129/pt/bumerangue-de-navalha Score:  8  Qtd Paginas= 25073
80 - https://scryfall.com/card/wwk/129/ja/%E5%88%83%E3%81%AE%E3%83%96%E3%83%BC%E3%83%A1%E3%83%A9%E3%83%B3 Score:  8  Qtd Paginas= 25407
81 - https://scryfall.com/card/wwk/129/ru/%D0%B1%D1%83%D0%BC%D0%B5%D1%80%D0%B0%D0%BD%D0%B3-%D0%BB%D0%B5%D0%B7%D0%B2%D0%B8%D0%B5 Score:  8  Qtd Paginas= 25741
82 - https://scryfall.com/card/wwk/129/zhs/%E9%94%8B%E5%88%83%E5%9B%9E%E5%8A%9B%E9%95%96 Score:  8  Qtd Paginas= 26075
83 - https://scryfall.com/contact?problem_uri=https%3A%2F%2Fscryfall.com%2Fcard%2Fwwk%2F129%2Frazor-boomerang&title=Problem+with+Razor+Boomerang+%28WWK%29&topic=card Score:  8  Qtd Paginas= 26243
84 - https://scryfall.com/card/wwk/129/razor-boomerang#rulings Score:  8  Qtd Paginas= 26577
85 - https://scryfall

121 - https://scryfall.com/contact?problem_uri=https%3A%2F%2Fscryfall.com%2Fcard%2Fsoi%2F262%2Fzhs%2F%25E7%258E%25BB%25E7%2592%2583%25E7%25A2%258E%25E7%2589%2587&title=Problem+with+Shard+of+Broken+Glass+%28SOI%29&topic=card Score:  8  Qtd Paginas= 31982
122 - https://scryfall.com/contact?problem_uri=https%3A%2F%2Fscryfall.com%2Fcard%2Fsoi%2F262%2Fzht%2F%25E7%258E%25BB%25E7%2592%2583%25E7%25A2%258E%25E7%2589%2587&title=Problem+with+Shard+of+Broken+Glass+%28SOI%29&topic=card Score:  8  Qtd Paginas= 32068
123 - https://scryfall.com/contact?problem_uri=https%3A%2F%2Fscryfall.com%2Fcard%2Fdtk%2F239%2Fes%2Faplastapuertas&title=Problem+with+Gate+Smasher+%28DTK%29&topic=card Score:  8  Qtd Paginas= 32154
124 - https://scryfall.com/card/dtk/239/es/aplastapuertas#rulings Score:  8  Qtd Paginas= 32498
125 - https://scryfall.com/contact?problem_uri=https%3A%2F%2Fscryfall.com%2Fcard%2Fdtk%2F239%2Ffr%2Ffracasseur-de-porte&title=Problem+with+Gate+Smasher+%28DTK%29&topic=card Score:  8  Qtd Paginas= 3

165 - https://scryfall.com/card/wwk/20/zhs/%E9%94%BB%E7%9F%B3%E7%A7%98%E6%95%99%E5%BE%92#rulings Score:  8  Qtd Paginas= 40306
166 - https://scryfall.com/contact?problem_uri=https%3A%2F%2Fscryfall.com%2Fcard%2Fprm%2F62499%2Fstoneforge-mystic&title=Problem+with+Stoneforge+Mystic+%28PRM%29&topic=card Score:  8  Qtd Paginas= 40396
167 - https://scryfall.com/card/prm/62499/stoneforge-mystic#rulings Score:  8  Qtd Paginas= 40752
168 - https://scryfall.com/contact?problem_uri=https%3A%2F%2Fscryfall.com%2Fcard%2Fpgpx%2F2016%2Fstoneforge-mystic&title=Problem+with+Stoneforge+Mystic+%28PGPX%29&topic=card Score:  8  Qtd Paginas= 40838
169 - https://scryfall.com/card/pgpx/2016/stoneforge-mystic#rulings Score:  8  Qtd Paginas= 41197
170 - https://scryfall.com/contact?problem_uri=https%3A%2F%2Fscryfall.com%2Fcard%2Ftd0%2FB11%2Fstoneforge-mystic&title=Problem+with+Stoneforge+Mystic+%28TD0%29&topic=card Score:  8  Qtd Paginas= 41278
171 - https://scryfall.com/card/td0/B11/stoneforge-mystic#rulings Sco

228 - https://scryfall.com/card/eld/21/lonesome-unicorn-rider-in-need Score:  8  Qtd Paginas= 61668
229 - https://scryfall.com/card/eld/22/mysterious-pathlighter Score:  8  Qtd Paginas= 61995
230 - https://scryfall.com/card/eld/23/outflank Score:  8  Qtd Paginas= 62328
231 - https://scryfall.com/card/eld/24/prized-griffin Score:  8  Qtd Paginas= 62658
232 - https://scryfall.com/card/eld/25/rally-for-the-throne Score:  8  Qtd Paginas= 62991
233 - https://scryfall.com/card/eld/26/realm-cloaked-giant-cast-off Score:  8  Qtd Paginas= 63354
234 - https://scryfall.com/card/eld/27/righteousness Score:  8  Qtd Paginas= 63903
235 - https://scryfall.com/card/eld/28/shepherd-of-the-flock-usher-to-safety Score:  8  Qtd Paginas= 64224
236 - https://scryfall.com/card/eld/28/it/pastore-del-gregge-condurre-alla-salvezza Score:  10  Qtd Paginas= 64562
237 - https://scryfall.com/contact?problem_uri=https%3A%2F%2Fscryfall.com%2Fcard%2Feld%2F28%2Fit%2Fpastore-del-gregge-condurre-alla-salvezza&title=Proble

...

247 - http://www.mtgmintcard.com/images/mtg/singles/m11/eng-foil/SORCERERS-STRONGBOX.jpg Score:  22  Qtd Paginas= 409918
248 - http://www.mtgmintcard.com/mtg/singles/m11/eng-foil/sorcerers-strongbox Score:  22  Qtd Paginas= 413122
249 - http://www.mtgmintcard.com/images/mtg/singles/m11/eng-foil/INFANTRY-VETERAN.jpg Score:  22  Qtd Paginas= 413121
250 - http://www.mtgmintcard.com/mtg/singles/m11/eng-foil/infantry-veteran Score:  22  Qtd Paginas= 416365
251 - http://www.mtgmintcard.com/images/mtg/singles/m11/eng-foil/WALL-OF-FROST.jpg Score:  22  Qtd Paginas= 416364
252 - http://www.mtgmintcard.com/mtg/singles/m11/eng-foil/wall-of-frost Score:  22  Qtd Paginas= 419815
253 - http://www.mtgmintcard.com/images/mtg/singles/m11/eng-foil/WATER-SERVANT.jpg Score:  22  Qtd Paginas= 419814
254 - http://www.mtgmintcard.com/mtg/singles/m11/eng-foil/water-servant Score:  22  Qtd Paginas= 423098
255 - http://www.mtgmintcard.com/mtg/singles/m10/chi-foil/wall-of-bone Score:  22  Qtd Paginas= 426082
256

323 - http://www.mtgmintcard.com/mtg/singles/m10/eng-foil/sparkmage-apprentice Score:  22  Qtd Paginas= 542252
324 - http://www.mtgmintcard.com/images/mtg/singles/m10/eng-foil/DISORIENT.jpg Score:  22  Qtd Paginas= 542251
325 - http://www.mtgmintcard.com/mtg/singles/m10/eng-foil/disorient Score:  22  Qtd Paginas= 545455
326 - http://www.mtgmintcard.com/images/mtg/singles/m12/chi-foil/AZURE-MAGE.jpg Score:  22  Qtd Paginas= 545228
327 - http://www.mtgmintcard.com/mtg/singles/m12/chi-foil/azure-mage Score:  22  Qtd Paginas= 548112
328 - http://www.mtgmintcard.com/images/mtg/singles/m12/chi-foil/GLADECOVER-SCOUT.jpg Score:  22  Qtd Paginas= 548111
329 - http://www.mtgmintcard.com/mtg/singles/m12/chi-foil/gladecover-scout Score:  22  Qtd Paginas= 551355
330 - http://www.mtgmintcard.com/images/mtg/singles/m12/chi-foil/SCEPTER-OF-EMPIRES.jpg Score:  22  Qtd Paginas= 551354
331 - http://www.mtgmintcard.com/mtg/singles/m12/chi-foil/scepter-of-empires Score:  22  Qtd Paginas= 554557
332 - http:

398 - http://www.mtgmintcard.com/mtg/singles/8ed/eng-foil/lesser-gargadon Score:  22  Qtd Paginas= 667574
399 - http://www.mtgmintcard.com/images/mtg/singles/8ed/eng-foil/ORCISH-ARTILLERY.jpg Score:  22  Qtd Paginas= 667573
400 - http://www.mtgmintcard.com/mtg/singles/8ed/eng-foil/orcish-artillery Score:  22  Qtd Paginas= 670815
401 - http://www.mtgmintcard.com/images/mtg/singles/8ed/eng-foil/DEMOLISH.jpg Score:  22  Qtd Paginas= 670814
402 - http://www.mtgmintcard.com/mtg/singles/8ed/eng-foil/demolish Score:  22  Qtd Paginas= 674260
403 - http://www.mtgmintcard.com/mtg/singles/m10/chi-foil/bog-wraith Score:  22  Qtd Paginas= 676885
404 - http://www.mtgmintcard.com/images/mtg/singles/8ed/eng-foil/BOG-WRAITH.jpg Score:  22  Qtd Paginas= 676884
405 - http://www.mtgmintcard.com/mtg/singles/8ed/eng-foil/bog-wraith Score:  22  Qtd Paginas= 680126
406 - http://www.mtgmintcard.com/images/mtg/singles/m10/eng-foil/KALONIAN-BEHEMOTH.jpg Score:  22  Qtd Paginas= 680107
407 - http://www.mtgmintcar

473 - http://www.mtgmintcard.com/mtg/singles/8ed/eng-foil/glorious-anthem Score:  22  Qtd Paginas= 792032
474 - http://www.mtgmintcard.com/images/mtg/singles/10e/eng-foil/FLAMEWAVE-INVOKER.jpg Score:  22  Qtd Paginas= 792028
475 - http://www.mtgmintcard.com/mtg/singles/10e/eng-foil/flamewave-invoker Score:  22  Qtd Paginas= 795192
476 - http://www.mtgmintcard.com/images/mtg/singles/10e/eng-foil/THRULL-SURGEON.jpg Score:  22  Qtd Paginas= 795191
477 - http://www.mtgmintcard.com/mtg/singles/10e/eng-foil/thrull-surgeon Score:  22  Qtd Paginas= 798354
478 - http://www.mtgmintcard.com/images/mtg/singles/10e/eng-foil/DELUGE.jpg Score:  22  Qtd Paginas= 798353
479 - http://www.mtgmintcard.com/mtg/singles/10e/eng-foil/deluge Score:  22  Qtd Paginas= 801476
480 - http://www.mtgmintcard.com/images/mtg/singles/10e/eng-foil/SPIRIT-WEAVER.jpg Score:  22  Qtd Paginas= 801475
481 - http://www.mtgmintcard.com/mtg/singles/10e/eng-foil/spirit-weaver Score:  22  Qtd Paginas= 804678
482 - http://www.mtgmi

548 - http://www.mtgmintcard.com/mtg/singles/mm3/eng-foil/blood-moon Score:  22  Qtd Paginas= 910254
549 - http://www.mtgmintcard.com/images/mtg/singles/mp1/eng-foil/DIABOLIC-EDICT.jpg Score:  22  Qtd Paginas= 910253
550 - http://www.mtgmintcard.com/mtg/singles/mp1/eng-foil/diabolic-edict Score:  22  Qtd Paginas= 913165
551 - http://www.mtgmintcard.com/images/mtg/singles/mp1/eng-foil/FORCE-OF-WILL.jpg Score:  22  Qtd Paginas= 913161
552 - http://www.mtgmintcard.com/mtg/singles/mp1/eng-foil/force-of-will Score:  22  Qtd Paginas= 916700
553 - http://www.mtgmintcard.com/images/mtg/singles/mp1/eng-foil/THREADS-OF-DISLOYALTY.jpg Score:  22  Qtd Paginas= 916699
554 - http://www.mtgmintcard.com/mtg/singles/mp1/eng-foil/threads-of-disloyalty Score:  22  Qtd Paginas= 919611
555 - http://www.mtgmintcard.com/images/mtg/singles/mp1/eng-foil/MAELSTROM-PULSE.jpg Score:  22  Qtd Paginas= 919610
556 - http://www.mtgmintcard.com/mtg/singles/mp1/eng-foil/maelstrom-pulse Score:  22  Qtd Paginas= 922632
5

### Dados da Heurística

In [ ]:
data = pd.DataFrame(Dados,columns=['Número','Link','Site'])
data.head()

#### Dados - csv

### Dados de Tempo da Heurística

In [12]:
TempoDominio

[]

In [13]:
timeData = pd.DataFrame(TempoDominio,columns=['Dominio','Tempo'])
timeData.head()

,Dominio,Tempo


#### Média Tempo-Download

In [14]:
print('Média Tempo-Download em Segundos: ',timeData['Tempo'].mean())
print('Média Tempo-Download em Minutos: ',timeData['Tempo'].mean()/60)
print('Média Tempo-Download em Horas: ',(timeData['Tempo'].mean()/60)/60)

Média Tempo-Download em Segundos:  nan
Média Tempo-Download em Minutos:  nan
Média Tempo-Download em Horas:  nan


#### DataFrame em Minutos

In [15]:
# Convertendo para minutos
timeData['Tempo'] = timeData['Tempo'].apply(lambda x: x / 60)
timeData

,Dominio,Tempo


#### DataFrame em Horas

In [16]:
# Convertendo para horas
timeData['Tempo'] = timeData['Tempo'].apply(lambda x: x / 60)
timeData

,Dominio,Tempo


### Tempo-Download

In [17]:
# Tempo de download dos anteriores - tempo em horas
timeData.to_csv('CrawlerHeuristicaTime.csv',index_label=False)
timeData

,Dominio,Tempo


#### * Teste

In [66]:
#request - usando o content type
codigo_fonte = rq.get(Dominio[0],headers={'content-type': 'text/html'})
# texto do codigo_fonte.
texto = codigo_fonte.text
soup = BeautifulSoup(texto,'html.parser')

# lista de tags que podem conter links para listas de cartas.
listasLinks = soup.findAll(re.compile(r'li|ul|td|tr'))

#print(listasLinks)
teste = []
for lista in soup.findAll(re.compile(r'li|ul|td|tr')):
    for link in lista.findAll('a',href = True):
        href = link.get('href')
        #texto
        urltexto = link.text
        
        #parents
        #print(link.parent.text.split('\n')
        parent_text = re.sub('[\s+]', '', link.parent.text)
        if len(parent_text) > 200:
            parent_text = ''
        if href not in teste:
            teste.append((href,urltexto,parent_text))
                         
                         
count = 0
for item in teste:
    count = count + 1
    print(item[2])

Home
SellUsCards
FAQ&Policies
Specials
Multi-Search
RetailStore
Home
SellUsCards
FAQ&Policies
Specials
Multi-Search
RetailStore






Home
SellusCards
FAQ&Policies
Specials
Multi-Search
RetailStore
Home
SellusCards
FAQ&Policies
Specials
Multi-Search
RetailStore






Oko,ThiefofCrowns$39.95
OnceUponaTime$13.95
Garruk,CursedHuntsman$11.95
GildedGoose$6.95
TheCircleofLoyalty$3.95
Oko,ThiefofCrowns$39.95
OnceUponaTime$13.95
Garruk,CursedHuntsman$11.95
GildedGoose$6.95
TheCircleofLoyalty$3.95
ModernHorizonsBoosterBox$219.99
CoreSet2020BoosterBox$99.99
CoreSet2020Bundle$39.95
ModernHorizonsBoosterPack$7.99
CoreSet2020BoosterPack$3.99
ModernHorizonsBoosterBox$219.99
CoreSet2020BoosterBox$99.99
CoreSet2020Bundle$39.95
ModernHorizonsBoosterPack$7.99
CoreSet2020BoosterPack$3.99
TheRiseofQueensdale$64.99
Expancity$59.99
TrekkingtheNationalParks$49.99
Sagrada$44.99
TheRiseofQueensdale$64.99
Expancity$59.99
TrekkingtheNationalParks$49.99
Sagrada$44.99
MonsterRocks:28MagicTheGatheringTokendiceColle

CommanderAnthologyVolumeII
Commander'sArsenal(Foil)
CommanderSingles(Oversized)
ConspiracySetsConspiracyConspiracy:TaketheCrown
Conspiracy
Conspiracy:TaketheCrown
Conspiracy
Conspiracy:TaketheCrown
Conspiracy
Conspiracy:TaketheCrown
DuelDecksDuelDecks:Ajanivs.NicolBolasDuelDecks:AnthologyDuelDecks:Blessedvs.CursedDuelDecks:Divinevs.DemonicDuelDecks:Elspethvs.KioraDuelDecks:Elspethvs.TezzeretDuelDecks:Elvesvs.GoblinsDuelDecks:Elvesvs.InventorsDuelDecks:Garrukvs.LilianaDuelDecks:Heroesvs.MonstersDuelDecks:Izzetvs.GolgariDuelDecks:Jacevs.ChandraDuelDecks:Jacevs.VraskaDuelDecks:JapaneseJacevs.ChandraDuelDecks:Knightsvs.DragonsDuelDecks:Merfolkvs.GoblinsDuelDecks:Mindvs.MightDuelDecks:Nissavs.ObNixilisDuelDecks:Phyrexiavs.theCoalitionDuelDecks:Sorinvs.TibaltDuelDecks:Speedvs.CunningDuelDecks:Venservs.KothDuelDecks:Zendikarvs.Eldrazi
DuelDecks:Ajanivs.NicolBolas
DuelDecks:Anthology
DuelDecks:Blessedvs.Cursed
DuelDecks:Divinevs.Demonic
DuelDecks:Elspethvs.Kiora
DuelDecks:Elspethvs.Tezzeret
Du

DarkLegends
ExclusivePack
PremiumPack1
PremiumPack2
RetroPack1
RetroPack2
DuelistSaga
AnniversaryPack
VideoGamePromos
WC4GBAPromos
2018AdventCalendar
2019Yu-Gi-OhRisingRampageBattlesofLegend:Hero'sRevengeSpeedDuel:AttackfromtheDeepDarkNeostormStructureDeck:OrderoftheSpellcastersDuelPowerSpeedDuel:ArenaofLostSoulsTheInfinityChasersStructureDeck:SoulburnerSavageStrikeLegendaryDuelists:SistersoftheRose2019GoldSarcophagusTinPromosDuelDevastatorLegendaryDuelists:ImmortalDestiny
RisingRampage
BattlesofLegend:Hero'sRevenge
SpeedDuel:AttackfromtheDeep
DarkNeostorm
StructureDeck:OrderoftheSpellcasters
DuelPower
SpeedDuel:ArenaofLostSouls
TheInfinityChasers
StructureDeck:Soulburner
SavageStrike
LegendaryDuelists:SistersoftheRose
2019GoldSarcophagusTinPromos
DuelDevastator
LegendaryDuelists:ImmortalDestiny
RisingRampage
BattlesofLegend:Hero'sRevenge
SpeedDuel:AttackfromtheDeep
DarkNeostorm
StructureDeck:OrderoftheSpellcasters
DuelPower
SpeedDuel:ArenaofLostSouls
TheInfinityChasers
StructureDeck:S